<a href="https://colab.research.google.com/github/miinsss/Capstone-Project/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libaries

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import recall_score
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.utils import resample

# Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the contract_classification.csv file
contract_class_path = '/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/data_all_unique_values.csv'
contract_class = pd.read_csv(contract_class_path)



# Display the first few rows to verify content
print(contract_class.head())

<ipython-input-3-e723c9e46e08>:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  contract_class = pd.read_csv(contract_class_path)


   Unnamed: 0    ANO_SID CORPORATE_DEVISION           Bundesland    Typ  \
0           0  4114028.0                VHV  Nordrhein-Westfalen  Stadt   
1           1  4114039.0                VHV  Nordrhein-Westfalen  Stadt   
2           2  4114045.0                VHV  Nordrhein-Westfalen  Stadt   
3           3  4114049.0                VGV  Nordrhein-Westfalen  Stadt   
4           4  4114055.0                VHV  Nordrhein-Westfalen  Kreis   

   ORTPLZ CONSTRACTION_DESIGN  CONSTRUCTION_YEAR        WFL ZONE  ...  \
0   42109      NORMAL_VENTURE        1967.565648   69.00000  2.0  ...   
1   42277      NORMAL_VENTURE        1967.565648   65.00000  4.0  ...   
2   42389      DESIGN_CLASS_I        1967.565648   75.00000  1.0  ...   
3   42277      NORMAL_VENTURE        1967.565648  106.24368  NaN  ...   
4   42553      NORMAL_VENTURE        1967.565648  119.00000  2.0  ...   

   DRAIN_PIPE_INSURED  PRODUCTLINE PRIOR_DAMAGES  UVV-KZ  UNDERWRITER  YEAR  \
0                   0        So

In [4]:
print(contract_class['DAMAGE'].value_counts())


na_counts = contract_class.isna().sum()

print(na_counts)

DAMAGE
0    9330675
1     135686
Name: count, dtype: int64
Unnamed: 0                     0
ANO_SID                        0
CORPORATE_DEVISION             0
Bundesland                     0
Typ                            0
ORTPLZ                         0
CONSTRACTION_DESIGN            0
CONSTRUCTION_YEAR              0
WFL                            0
ZONE                      777969
TYPE_OF_DEDUCTIBLE             0
DRAIN_PIPE_INSURED             0
PRODUCTLINE                    0
PRIOR_DAMAGES                  0
UVV-KZ                         0
UNDERWRITER                    0
YEAR                           0
DAMAGE_HEAVY_RAIN_ZONE         0
LONGITUDE                      0
LATITUDE                       0
DAMAGE                         0
dtype: int64


In [5]:
# Show the number of entries for each unique value in ANO_SID
zone_counts = contract_class['ANO_SID'].value_counts()

print(zone_counts)

ANO_SID
6407311.0      22
11598817.0     22
5291031.0      22
12842117.0     22
23939555.0     22
               ..
40083039.0      1
58201225.0      1
58201226.0      1
142730256.0     1
145240911.0     1
Name: count, Length: 1386416, dtype: int64


# Models

## Preprocess the data

In [22]:
# Preprocessing from Francesca
# Clean data that is not needed for the model (or would be too hard to encode)
#contracts = contract_class.drop(['Unnamed: 0', 'SF-SYSTEM',  'ORTS-NAME', 'STRASSE', 'PARTY-ID', 'contract_year',
                            #'Kreis', 'DAMAGE_FLOOD_ZONE', 'SUM_INSURED', 'PIPE_PREMIUM_AMOUNT'], axis=1)
contracts = contract_class.dropna()

# Convert all zone values to string (to reduce amount of unique values)
print('Original unique values:', contracts['ZONE'].nunique())
contracts['ZONE'] = contracts['ZONE'].astype('str')
print('New unique values:', contracts['ZONE'].nunique())

# Prepare binary variables
columns_to_encode = contracts.select_dtypes(include=['object']).columns
df_binary = pd.get_dummies(contracts, columns=columns_to_encode, drop_first=True)
df_binary.info()

print(contract_class['DAMAGE'].value_counts())

# Separate features and target variable
X = df_binary.drop(columns=['DAMAGE', 'ANO_SID' ])
y = df_binary['DAMAGE']


Original unique values: 32


<ipython-input-22-970d7c867732>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts['ZONE'] = contracts['ZONE'].astype('str')


New unique values: 23
<class 'pandas.core.frame.DataFrame'>
Index: 8688392 entries, 0 to 9466360
Data columns (total 72 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   Unnamed: 0                                     int64  
 1   ANO_SID                                        float64
 2   ORTPLZ                                         int64  
 3   CONSTRUCTION_YEAR                              float64
 4   WFL                                            float64
 5   TYPE_OF_DEDUCTIBLE                             int64  
 6   DRAIN_PIPE_INSURED                             int64  
 7   PRIOR_DAMAGES                                  int64  
 8   UVV-KZ                                         int64  
 9   YEAR                                           int64  
 10  DAMAGE_HEAVY_RAIN_ZONE                         float64
 11  LONGITUDE                                      float64
 12  LATITUDE                 

In [ ]:
# Choose columns to encode to binary variables

columns_to_encode = ['CORPORATE_DEVISION','CONSTRACTION_DESIGN', 'PRODUCTLINE', 'UNDERWRITER', 'ZONE']

# Create Binary Variables
features_binary = pd.get_dummies(initial_features_clean, columns=columns_to_encode, drop_first=True)

print(features_binary.head())

     ANO_SID  ORTPLZ  CONSTRUCTION_YEAR    WFL  DRAIN_PIPE_INSURED  \
0  4114028.0   42109        1967.565648   69.0                   0   
1  4114039.0   42277        1967.565648   65.0                   0   
2  4114045.0   42389        1967.565648   75.0                   0   
4  4114055.0   42553        1967.565648  119.0                   0   
5  4114057.0   42113        1967.565648  100.0                   0   

   PRIOR_DAMAGES  UVV-KZ  DAMAGE  DAMAGE_FLOOD_ZONE  DAMAGE_HEAVY_RAIN_ZONE  \
0              0       1       0                1.0                1.727273   
1              0       1       0                1.0                1.931034   
2              0       1       0                1.0                1.962963   
4              0       1       0                1.0                1.768000   
5              0       1       0                1.0                1.824324   

   ...  ZONE_4  ZONE_4.0  ZONE_5  ZONE_5.0  ZONE_6  ZONE_6.0  ZONE_7  \
0  ...   False     False   False

## Random Forest

In [18]:
# Define feature and target

X = df_binary.drop(columns=['ANO_SID','DAMAGE' ])
y = df_binary['DAMAGE']

# Split into test and training

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [19]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)

# Training the classifier
rf_classifier.fit(X_train, y_train)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)

# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))
# With undersampling accuracy gets worse, but we detect more TRUE Positives. Still way more false positive

Accuracy: 0.9891090356734472
Confusion Matrix:
[[1715513    1775]
 [  17150    3241]]
True Positives: 3241
False Positives: 1775
True Negatives: 1715513
False Negatives: 17150
              precision    recall  f1-score   support

           0       0.99      1.00      0.99   1717288
           1       0.65      0.16      0.26     20391

    accuracy                           0.99   1737679
   macro avg       0.82      0.58      0.62   1737679
weighted avg       0.99      0.99      0.99   1737679



In [20]:
# Feature-importnace analysis
feature_importance = rf_classifier.feature_importances_

# Display results
print("Feature Importance:")
for i, feature in enumerate(X_train.columns):
    print(f"{feature}: {feature_importance[i]}")

Feature Importance:
Unnamed: 0: 0.2611810248647393
ORTPLZ: 0.07838337165096272
CONSTRUCTION_YEAR: 0.0798196684123265
WFL: 0.05170992260575101
TYPE_OF_DEDUCTIBLE: 0.00010150798436302533
DRAIN_PIPE_INSURED: 0.0
PRIOR_DAMAGES: 0.004635085575754964
UVV-KZ: 0.002410800592780058
YEAR: 0.11593291199665239
DAMAGE_HEAVY_RAIN_ZONE: 0.06600636168778863
LONGITUDE: 0.13911226726399198
LATITUDE: 0.13771501386122004
CORPORATE_DEVISION_VGV: 0.007504421968788916
CORPORATE_DEVISION_VHV: 0.0032633061868500403
CORPORATE_DEVISION_W&W: 0.00017181664120910833
Bundesland_Bayern: 0.0010661678964717764
Bundesland_Berlin: 0.0011897438177455922
Bundesland_Brandenburg: 0.0003307938479711785
Bundesland_Bremen: 0.0002446898699122573
Bundesland_Hamburg: 0.0003170639748470795
Bundesland_Hessen: 0.0013655351634069644
Bundesland_Mecklenburg-Vorpommern: 0.00030011705046435684
Bundesland_Niedersachsen: 0.0012820918977853749
Bundesland_Nordrhein-Westfalen: 0.001541670710154834
Bundesland_Rheinland-Pfalz: 0.0013457662893001

## Random Forest with  Resampling

In [21]:
# Define feature and target
X = df_binary.drop(columns=['ANO_SID','DAMAGE'])
y = df_binary['DAMAGE']

# Split into test and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Resample the training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

# Print the shape of resampled data
print('Original dataset shape:', y_train.value_counts())
print('Resampled dataset shape:', y_resampled.value_counts())

Original dataset shape: DAMAGE
0    6870811
1      79902
Name: count, dtype: int64
Resampled dataset shape: DAMAGE
0    79902
1    79902
Name: count, dtype: int64


In [23]:
# Initializing Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=42)

# Training the classifier
rf_classifier.fit(X_resampled, y_resampled)

# Making probability predictions
y_pred_proba = rf_classifier.predict_proba(X_test)

KeyboardInterrupt: 

In [ ]:
# Evaluating the model

# Converting probabilities to class predictions
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generating confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Displaying confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Extracting TP, FP, TN, FN from the confusion matrix
TN = conf_matrix[0][0]
FP = conf_matrix[0][1]
FN = conf_matrix[1][0]
TP = conf_matrix[1][1]

# Printing TP, FP, TN, FN
print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.7983528057546572
Confusion Matrix:
[[1227367  308621]
 [   5652   16889]]
True Positives: 16889
False Positives: 308621
True Negatives: 1227367
False Negatives: 5652
              precision    recall  f1-score   support

           0       1.00      0.80      0.89   1535988
           1       0.05      0.75      0.10     22541

    accuracy                           0.80   1558529
   macro avg       0.52      0.77      0.49   1558529
weighted avg       0.98      0.80      0.88   1558529



## Random Forest with Rolling window and SMOTE

In [27]:
# Create a list of unique years in the data
years = sorted(df_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[df_binary['YEAR'] == train_year]
    y_train = y[df_binary['YEAR'] == train_year]
    X_test = X[df_binary['YEAR'] == test_year]
    y_test = y[df_binary['YEAR'] == test_year]

    ## Initialize the SMOTE resampler
    smote = SMOTE(random_state=42)

    # Resample the training data
    X_resampled, y_resampled =  smote.fit_resample(X_train, y_train)

    # Initialize and train Random Forest classifier
    rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
    rf_classifier.fit(X_resampled, y_resampled)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = contracts[contracts['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('combined_probabilities_all_years.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())



  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    745209
           1       0.08      0.04      0.05      7391

    accuracy                           0.99    752600
   macro avg       0.54      0.52      0.52    752600
weighted avg       0.98      0.99      0.98    752600

Confusion Matrix for year 2015:
[[742217   2992]
 [  7118    273]]
True Positives: 273
False Positives: 2992
True Negatives: 742217
False Negatives: 7118


 10%|█         | 1/10 [00:54<08:09, 54.41s/it]


KeyboardInterrupt: 

## Random Forest with roling window and k fold cross validation

In [28]:
# Create a list of unique years in the data
years = sorted(df_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Initialize the Stratified K-Fold Cross Validation
n_splits = 5  # You can adjust the number of splits
skf = StratifiedKFold(n_splits=n_splits)

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[df_binary['YEAR'] == train_year]
    y_train = y[df_binary['YEAR'] == train_year]
    X_test = X[df_binary['YEAR'] == test_year]
    y_test = y[df_binary['YEAR'] == test_year]

    # Perform Stratified K-Fold Cross Validation within the training year
    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

         # Fit your model here on X_train_fold and y_train_fold
        model = RandomForestClassifier(n_estimators=10, random_state=42)
        model.fit(X_train_fold, y_train_fold)

        # Validate your model here on X_val_fold and y_val_fold
        y_val_pred = model.predict(X_val_fold)
        fold_recall = recall_score(y_val_fold, y_val_pred)
        recall_scores.append(fold_recall)

    # Fit the model on the entire training set after cross-validation
    model.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = contracts[contracts['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:879: RuntimeWarning: invalid value encountered in divide
  proba /= len(self.estimators_)


Classification report for year 2015:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00    745209
           1       0.00      0.00      0.00      7391

    accuracy                           0.99    752600
   macro avg       0.50      0.50      0.50    752600
weighted avg       0.98      0.99      0.99    752600

Confusion Matrix for year 2015:
[[745209      0]
 [  7391      0]]
True Positives: 0
False Positives: 0
True Negatives: 745209
False Negatives: 7391


 10%|█         | 1/10 [01:22<12:21, 82.37s/it]


KeyboardInterrupt: 

## Balanced Random Forest with Roling window

In [30]:
# Create a list of unique years in the data
years = sorted(df_binary['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(len(years) - 1)):
    train_year = years[i]
    test_year = years[i + 1]

    # Split data into training and testing sets based on year
    X_train = X[df_binary['YEAR'] == train_year]
    y_train = y[df_binary['YEAR'] == train_year]
    X_test = X[df_binary['YEAR'] == test_year]
    y_test = y[df_binary['YEAR'] == test_year]



    # Initialize and train Random Forest classifier
    rf_classifier = BalancedRandomForestClassifier(n_estimators=50, random_state=42)
    rf_classifier.fit(X_train, y_train)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)



    # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Print recall scores
for year, score in recall_scores:
    print(f"Year: {year}, Recall Score: {score}")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py

Classification report for year 2015:


 10%|█         | 1/10 [00:14<02:13, 14.85s/it]

              precision    recall  f1-score   support

           0       1.00      0.70      0.83    745209
           1       0.02      0.74      0.05      7391

    accuracy                           0.70    752600
   macro avg       0.51      0.72      0.44    752600
weighted avg       0.99      0.70      0.82    752600

Confusion Matrix for year 2015:
[[525022 220187]
 [  1930   5461]]
True Positives: 5461
False Positives: 220187
True Negatives: 525022
False Negatives: 1930


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.para

KeyboardInterrupt: 

## Random Forest with expanding window and tuning

In [32]:
# Preprocessing from Francesca
# Clean data that is not needed for the model (or would be too hard to encode)
#contracts = contract_class.drop(['Unnamed: 0', 'SF-SYSTEM',  'ORTS-NAME', 'STRASSE', 'PARTY-ID', 'contract_year',
                            #'Kreis', 'DAMAGE_FLOOD_ZONE', 'SUM_INSURED', 'PIPE_PREMIUM_AMOUNT'], axis=1)
contracts = contract_class.dropna()

# Convert all zone values to string (to reduce amount of unique values)
print('Original unique values:', contracts['ZONE'].nunique())
contracts['ZONE'] = contracts['ZONE'].astype('str')
print('New unique values:', contracts['ZONE'].nunique())

# Prepare binary variables
columns_to_encode = contracts.select_dtypes(include=['object']).columns
df_binary = pd.get_dummies(contracts, columns=columns_to_encode, drop_first=True)
df_binary.info()

# Separate features and target variable
X = df_binary.drop(columns=['DAMAGE', 'ANO_SID' ])
y = df_binary['DAMAGE']

# Ensure indices are aligned
contracts = contracts.reset_index(drop=True)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

print(contract_class['DAMAGE'].value_counts())

Original unique values: 32


<ipython-input-32-194d3e2cacb7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts['ZONE'] = contracts['ZONE'].astype('str')


New unique values: 23
<class 'pandas.core.frame.DataFrame'>
Index: 8688392 entries, 0 to 9466360
Data columns (total 72 columns):
 #   Column                                         Dtype  
---  ------                                         -----  
 0   Unnamed: 0                                     int64  
 1   ANO_SID                                        float64
 2   ORTPLZ                                         int64  
 3   CONSTRUCTION_YEAR                              float64
 4   WFL                                            float64
 5   TYPE_OF_DEDUCTIBLE                             int64  
 6   DRAIN_PIPE_INSURED                             int64  
 7   PRIOR_DAMAGES                                  int64  
 8   UVV-KZ                                         int64  
 9   YEAR                                           int64  
 10  DAMAGE_HEAVY_RAIN_ZONE                         float64
 11  LONGITUDE                                      float64
 12  LATITUDE                 

### With rebalancing

In [31]:


# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [10],
    'max_depth': [10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
}

# Create a list of unique years in the data
years = sorted(contracts['YEAR'].unique())

# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []
feature_importances = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    train_indices = contracts['YEAR'].isin(train_years)
    test_indices = contracts['YEAR'] == test_year

    X_train = X[train_indices].reset_index(drop=True)
    y_train = y[train_indices].reset_index(drop=True)
    X_test = X[test_indices].reset_index(drop=True)
    y_test = y[test_indices].reset_index(drop=True)

    # Resample classes to ratio 1:5 in the training data
    train = pd.concat([X_train, y_train], axis=1)
    class_0 = train[train['DAMAGE'] == 0]
    class_1 = train[train['DAMAGE'] == 1]

    n_samples_0 = len(class_0) // 2  # Undersample class 0 to half its size
    n_samples_1 = n_samples_0 // 5   # Oversample class 1 to be 1/5 of the undersampled class 0

    # Resample class 0 (undersample)
    class_0_resampled = resample(class_0,
                                 replace=False,
                                 n_samples=n_samples_0,
                                 random_state=42)

    # Resample class 1 (oversample)
    class_1_resampled = resample(class_1,
                                 replace=True,
                                 n_samples=n_samples_1,
                                 random_state=42)

    # Combine the resampled classes
    train_resampled = pd.concat([class_0_resampled, class_1_resampled])
    X_train_resampled = train_resampled.drop(columns='DAMAGE')
    y_train_resampled = train_resampled['DAMAGE']

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)

    # Perform Grid Search with cross-validation
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2, scoring='recall')
    grid_search.fit(X_train_resampled, y_train_resampled)

    # Get the best model from Grid Search
    best_rf_classifier = grid_search.best_estimator_
    # Make predictions (probabilities)
    y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]



    # Get the feature importances
    feature_importances.append(rf_classifier.feature_importances)
    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Store true and predicted labels
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)

    # Get the best model from Grid Search
    best_rf_classifier = grid_search.best_estimator_

    # Make predictions (probabilities)
    y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for year {test_year}: {grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

     # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = contracts[contracts['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

       # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
#combined_all_years.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/Random_Forest_with_rebalancing_2.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())

# Calculate average feature importance across all models
average_feature_importances = np.mean(feature_importances, axis=0)

# Print feature importances
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': average_feature_importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

  0%|          | 0/10 [00:00<?, ?it/s]

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Best parameters for year 2015: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 10}
Classification report for year 2015:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99    745209
           1       0.10      0.17      0.12      7391

    accuracy                           0.98    752600
   macro avg       0.54      0.58      0.56    752600
weighted avg       0.98      0.98      0.98    752600

Confusion Matrix for year 2015:
[[733831  11378]
 [  6153   1238]]
True Positives: 1238
False Positives: 11378
True Negatives: 733831
False Negatives: 6153


  0%|          | 0/10 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Save the combined DataFrame to a CSV filelab Notebooks/Capstone_Data/Capstone/Data/Random_Forest_with_rebalancing_2.csv', index=False)
combined_all_years.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/Random_Forest_with_rebalancing_2.csv', index=False)

In [ ]:
# Calculate average feature importance across all models
average_feature_importances = np.mean(feature_importances, axis=0)

# Print feature importances
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': average_feature_importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

NameError: name 'feature_importances' is not defined

### Without rebalancing



In [33]:
# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [10, 20],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Create a list of unique years in the data
years = sorted(contracts['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    train_indices = contracts['YEAR'].isin(train_years)
    test_indices = contracts['YEAR'] == test_year

    X_train = X[train_indices].reset_index(drop=True)
    y_train = y[train_indices].reset_index(drop=True)
    X_test = X[test_indices].reset_index(drop=True)
    y_test = y[test_indices].reset_index(drop=True)

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)


    # Perform Grid Search with cross-validation
    grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                               cv=3, n_jobs=-1, verbose=2, scoring='recall')
    grid_search.fit(X_train, y_train)

    # Get the best model from Grid Search
    best_rf_classifier = grid_search.best_estimator_

    # Make predictions (probabilities)
    y_prob = best_rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)


    # Get the feature importances
    feature_importances.append(rf_classifier.feature_importances_)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))

    # Print the best parameters for the current year
    print(f"Best parameters for year {test_year}: {grid_search.best_params_}")

    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)

     # Create a DataFrame for the probabilities
    probabilities_df = pd.DataFrame({'Probability': y_prob})

    # Combine the probabilities with the original test data for the year
    combined_df = contracts[contracts['YEAR'] == test_year].copy()
    combined_df['Probability'] = probabilities_df['Probability'].values

    # Save the combined DataFrame to a CSV file
    combined_df.to_csv(f'probabilities_{test_year}.csv', index=False)

       # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)


# Combine all the CSV files into a single DataFrame
combined_all_years = pd.concat([pd.read_csv(f'probabilities_{year}.csv') for year in years[1:]])

# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/Random_Forest_without_rebalancing.csv', index=False)

print("Combined probabilities for all years:")
print(combined_all_years.head())

# Calculate average feature importance across all models
average_feature_importances = np.mean(feature_importances, axis=0)

# Print feature importances
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': average_feature_importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

  0%|          | 0/10 [00:00<?, ?it/s]

Fitting 3 folds for each of 54 candidates, totalling 162 fits


  0%|          | 0/10 [01:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Calculate average feature importance across all models
average_feature_importances = np.mean(feature_importances, axis=0)

# Print feature importances
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': average_feature_importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

In [ ]:
# Save the combined DataFrame to a CSV file
combined_all_years.to_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Data/Capstone/Data/Random_Forest_without_rebalancing.csv', index=False)

### Without tuning

In [35]:
# Create a list of unique years in the data
years = sorted(contracts['YEAR'].unique())
# Initialize lists to store results
recall_scores = []
all_y_true = []
all_y_pred = []
feature_importances = []


# Best parameters for the RandomForestClassifier
best_params = {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}

# Loop over each year for rolling window estimation
for i in tqdm(range(1, len(years))):
    test_year = years[i]
    train_years = years[:i]  # All years up to the year before test_year

    # Split data into training and testing sets based on year
    train_indices = contracts['YEAR'].isin(train_years)
    test_indices = contracts['YEAR'] == test_year

    X_train = X[train_indices].reset_index(drop=True)
    y_train = y[train_indices].reset_index(drop=True)
    X_test = X[test_indices].reset_index(drop=True)
    y_test = y[test_indices].reset_index(drop=True)

    # Initialize the Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42, **best_params)
    rf_classifier.fit(X_train, y_train)

# Get the feature importances
    feature_importances.append(rf_classifier.feature_importances_)

    # Make predictions (probabilities)
    y_prob = rf_classifier.predict_proba(X_test)[:, 1]

    # Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
    y_pred = (y_prob >= 0.5).astype(int)

    # Calculate accuracy for the current year
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate recall score
    recall = recall_score(y_test, y_pred)
    recall_scores.append((test_year, recall))



    # Print the classification report for each year
    print(f"Classification report for year {test_year}:")
    print(classification_report(y_test, y_pred))

    # Generating confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix for year {test_year}:")
    print(conf_matrix)

    # Extracting TP, FP, TN, FN from the confusion matrix
    TN = conf_matrix[0][0]
    FP = conf_matrix[0][1]
    FN = conf_matrix[1][0]
    TP = conf_matrix[1][1]

    # Printing TP, FP, TN, FN
    print("True Positives:", TP)
    print("False Positives:", FP)
    print("True Negatives:", TN)
    print("False Negatives:", FN)
    # Accumulate true and predicted values
    all_y_true.extend(y_test)
    all_y_pred.extend(y_pred)



       # Calculate and print the combined confusion matrix and classification report
combined_conf_matrix = confusion_matrix(all_y_true, all_y_pred)
print("Combined Confusion Matrix:")
print(combined_conf_matrix)

# Extract and print TP, FP, TN, FN from the combined confusion matrix
combined_TN = combined_conf_matrix[0][0]
combined_FP = combined_conf_matrix[0][1]
combined_FN = combined_conf_matrix[1][0]
combined_TP = combined_conf_matrix[1][1]

print("Combined True Positives:", combined_TP)
print("Combined False Positives:", combined_FP)
print("Combined True Negatives:", combined_TN)
print("Combined False Negatives:", combined_FN)

combined_class_report = classification_report(all_y_true, all_y_pred)
print("Combined Classification Report:")
print(combined_class_report)

# Calculate average feature importance across all models
average_feature_importances = np.mean(feature_importances, axis=0)

# Print feature importances
feature_names = X.columns
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': average_feature_importances
}).sort_values(by='Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)


  0%|          | 0/10 [00:00<?, ?it/s]

Classification report for year 2015:


 10%|█         | 1/10 [00:32<04:53, 32.56s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    745209
           1       0.21      0.03      0.05      7391

    accuracy                           0.99    752600
   macro avg       0.60      0.51      0.52    752600
weighted avg       0.98      0.99      0.99    752600

Confusion Matrix for year 2015:
[[744403    806]
 [  7177    214]]
True Positives: 214
False Positives: 806
True Negatives: 744403
False Negatives: 7177
Classification report for year 2016:


 20%|██        | 2/10 [01:40<07:09, 53.66s/it]

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    769344
           1       0.25      0.04      0.07      8184

    accuracy                           0.99    777528
   macro avg       0.62      0.52      0.53    777528
weighted avg       0.98      0.99      0.98    777528

Confusion Matrix for year 2016:
[[768346    998]
 [  7852    332]]
True Positives: 332
False Positives: 998
True Negatives: 768346
False Negatives: 7852


 20%|██        | 2/10 [03:28<13:55, 104.45s/it]


KeyboardInterrupt: 

In [ ]:
# Calculate total importance
total_importance = feature_importance_df['Importance'].sum()
total_importance

1.0